In [100]:
import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
from awsglue.dynamicframe import DynamicFrame
import re
from pyspark.sql.functions import col, when, last_day, to_date, sum as _sum
from pyspark.sql import functions as F

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [101]:
# Simula los argumentos como si los estuvieras pasando por CLI
sys.argv = [
    'glue_script.py',  # nombre ficticio del script, solo para cumplir con la forma
    '--JOB_NAME', 'ingresos_2r',
    '--PAIS', '2',
    '--ANO', '2025',
    '--MES', '1'
]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [102]:
# Obtener argumentos del job
args = getResolvedOptions(sys.argv, ['JOB_NAME', 'PAIS', 'ANO', 'MES'])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [103]:
# Inicializar GlueContext y Job
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)
job.init(args['JOB_NAME'], args)

logger = glueContext.get_logger()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [104]:
def select_table_from_rd(table):
    node = glueContext.create_dynamic_frame.from_options(
    connection_type="redshift",
    connection_options={
        "redshiftTmpDir": "s3://aws-glue-assets-575238426438-us-east-1/temporary/",
        "useConnectionProperties": "true",
        "dbtable": table,
        "connectionName": "conexion-redshift"
    },
    transformation_ctx="node"
    )
    return node

def select_query_from_rd(query):
    node = glueContext.create_dynamic_frame.from_options(
        connection_type="redshift",
        connection_options={
            "sampleQuery": query,
            "redshiftTmpDir": "s3://aws-glue-assets-575238426438-us-east-1/temporary/",
            "useConnectionProperties": "true",
            "connectionName": "conexion-redshift"
        },
        transformation_ctx="node"
    )
    return node


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [105]:
pais = args['PAIS'].zfill(2)
ano = args['ANO']
mes = args['MES'].zfill(2)
esquema_pais = {
    '01': 'silver',
    '02': 'silver_pan'
}
empresa_pais = {
    '01': 'JA',
    '02': 'JP'
}

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# QUERYS

In [106]:
query_ingresos = f"""
WITH param AS (
  SELECT
    valor,
    grupo
  FROM bronze_col.parametros_r
)
select 
	FACTURA,
    CUE as CUENTA,
    UNIDAD,
    INTESES_DIRECTOS,
    CUOTA_MANEJO,
    SEGURO,
    VAL_SEGURO,
    --1 as "temp",
    (INTESES_DIRECTOS + CUOTA_MANEJO + SEGURO + VAL_SEGURO) as INGRESOS_MOD_FNRO,
    C_EMP
from (
	select 
		FACTURA,
	    CUE,
	    UNIDAD,
	    Sum(Int_Dir * tipo_valor) * -1  as INTESES_DIRECTOS,
	    Sum(Cm_Directo * tipo_valor) * -1 as CUOTA_MANEJO,
	    Sum(seguro * tipo_valor) * -1 as SEGURO,
	    sum(Val_Seguro * tipo_valor) * -1 as VAL_SEGURO,
        C_EMP
	from (
	    	-----------------------------------------------------------------------------------------
						--                     CARTERA                     --
			-----------------------------------------------------------------------------------------
	        (
	        	SELECT 
			        KC.CUE,
			        KC.documento_apl FACTURA,
			        KC.T_CRE,
			        KC.C_AGR,
			        ENTIDAD,
			        CASTIGO,
			        case
			        	when OPERACION in (SELECT valor FROM param WHERE grupo = 'TVAL_KARDEX_{pais}') then 1
			        	else -1
			        end tipo_valor,
			        SUM(KC.INTERES_MORA) Int_Mora,
				    SUM(KC.HONORARIOS) / (SELECT max(cast(valor as float)) FROM param WHERE grupo = 'PORC_HONORARIO') Honorarios,
			        SUM(KC.VAL_SEGURO) Val_Seguro,
			        SUM(KC.GAS_COB) - SUM(KC.IVA_GAS_COB) Gasto_Cobranza,
			        SUM(KC.INTERES) - SUM(KC.INTERES_CAU) Int_Dir,
			        (SUM(KC.CUOTA_MANEJO)
			        	- SUM(KC.IVA_CUOTA_MANEJO)
						+ SUM(KC.AVAL)
						- SUM(KC.IVA_AVAL)
						- SUM(KC.CONDONACION)
						+ SUM(KC.IVA_CONDONACION))
					- (SUM(KC.CUOTA_MANEJO_CAU)
						- SUM(KC.IVA_CUO_MANEJO_CAU)
						- SUM(KC.CONDONACION_CAU)
						+ SUM(KC.IVA_CONDONACION_CAU)) CM_Directo,
					SUM(KC.SEGURO) Seguro,
			        LAST_DAY(KC.FECHA_CREACION) UNIDAD,
                    KC.C_EMP
		        FROM 
		        	{esquema_pais.get(pais)}.kardex_cartera KC
		        WHERE 
			        KC.C_EMP = '{empresa_pais.get(pais)}'
			        AND KC.TIPO_MOV in (SELECT valor FROM param WHERE grupo = 'TMOV_KARDEX_{pais}')
			        AND KC.FECHA_CREACION >= to_date('{ano}-{mes}-01', 'yyyy-mm-dd')
			        AND KC.FECHA_CREACION <= last_day(to_date('{ano}-{mes}-01', 'yyyy-mm-dd'))
			        AND KC.ENTIDAD in (SELECT valor FROM param WHERE grupo = 'ENT_KARDEX_{pais}')  
		        GROUP BY 
			        KC.CUE,
			        KC.documento_apl,
			        KC.T_CRE,
			        KC.C_AGR,
			        ENTIDAD,
			        CASTIGO,
			        OPERACION,
			        KC.FECHA_CREACION,
                    KC.C_EMP
	        )  
	     	-- ***************************
	  	 	UNION ALL        
	   		-----------------------------------------------------------------------------------------
						--                     CIERRE CAUSACION                     --
			-----------------------------------------------------------------------------------------
	    	(	
				select
					CIC.CUE,
					CIC.N_ORI,
					CIC.T_CRE,
					CIC.C_AGR,
					CIC.ENTIDAD,
					null,
					case
						when CIC.TIPO_MOV in (SELECT valor FROM param WHERE grupo = 'TVAL_CIERRE_{pais}') then 1
						else -1
					end tipo_valor,
					0 Int_Mora,
					0 Honorarios,
					0 Val_Seguro,
					0 Gasto_Cobranza,
					sum(CIC.INTERES) Int_Dir,
					(
						sum(CIC.CUOTA_MANEJO) -
						sum(CIC.IVA_CUOTA_MANEJO) -
						sum(CIC.CONDONACION) +
						sum(CIC.IVA_CONDONACION)
					) CM_Directo,
					0 seguro,
					LAST_DAY(TO_DATE(CIC.PER, 'YYYYMM')) UNIDAD,
                    CIC.C_EMP
				from
					{esquema_pais.get(pais)}.CIERRE_CAUSACION CIC
				Where
					CIC.C_EMP = '{empresa_pais.get(pais)}'
					and CIC.PER = '{ano}{mes}'
					--and CIC.PER <= '202512'
					and TIPO_MOV in (SELECT valor FROM param WHERE grupo = 'TMOV_CIERRE_{pais}')
				group by
					CIC.C_AGR,
					CIC.T_CRE,
					CIC.CUE,
					CIC.TIPO_MOV,
					CIC.N_ORI,
					CIC.ENTIDAD,
					CIC.PER,
                    CIC.C_EMP
	        	)
	    -- ***************************
		)    
	group by 
		FACTURA,
		CUE,
		UNIDAD,
        C_EMP
)
"""

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [107]:
ingresos_node = select_query_from_rd(query_ingresos) 
ingresos_df = ingresos_node.toDF()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [108]:
ingresos_df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------+----------+----------------+------------+------+----------+-----------------+-----+
|factura|cuenta|    unidad|inteses_directos|cuota_manejo|seguro|val_seguro|ingresos_mod_fnro|c_emp|
+-------+------+----------+----------------+------------+------+----------+-----------------+-----+
|  91062| 22952|2025-01-31|           63.89|        0.00|  0.00|      3.01|            66.90|   JP|
|  53632| 59997|2025-01-31|           25.80|        0.00|  0.00|      1.56|            27.36|   JP|
|  10164|   288|2025-01-31|           37.83|        0.00|  0.00|     15.03|            52.86|   JP|
| 174875| 63112|2025-01-31|            6.79|        0.00|  0.00|      0.77|             7.56|   JP|
| 176257|   299|2025-01-31|           88.52|        0.00|  0.00|      9.50|            98.02|   JP|
|  23049| 55457|2025-01-31|           33.20|        0.00|  0.00|      0.00|            33.20|   JP|
|  82894| 21222|2025-01-31|            3.68|        0.00|  0.00|      0.00|             3.68|   JP|


In [109]:
ingresos_node_w = glueContext.write_dynamic_frame.from_options(
    frame=ingresos_node,
    connection_type="redshift",
    connection_options={
        "postactions": """
        """,
        "redshiftTmpDir": "s3://aws-glue-assets-575238426438-us-east-1/temporary/",
        "useConnectionProperties": "true",
        "dbtable": "cf_processing.ingresos_2r",
        "connectionName": "conexion-redshift",
        "preactions": f"""
            DELETE FROM cf_processing.ingresos_2r
            where unidad = last_day(to_date('{ano}-{mes}','yyyy-mm'))
            and c_emp = '{empresa_pais.get(pais)}';
        """
    },
    transformation_ctx="ingresos_node_w"
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…